# Milvus demo

## References

Source:
- https://milvus.io/docs/quickstart.md

### Create a database

In [ ]:
import os
from pymilvus import MilvusClient, model
from openai import AsyncAzureOpenAI
from dotenv import load_dotenv

load_dotenv()
ENDPOINT = os.getenv("ENDPOINT")
API_KEY = os.getenv("API_KEY")
API_VERSION = os.getenv("API_VERSION") or '2024-02-15-preview'
CHAT_MODEL = os.getenv("CHAT_MODEL")
EMB_MODEL = os.getenv("EMB_MODEL")

client = MilvusClient("milvus_demo_oai.db")
oclient = AsyncAzureOpenAI(azure_endpoint=ENDPOINT,api_key=API_KEY,api_version=API_VERSION)

In [ ]:
async def embed(text:str,model:str):
    res = await oclient.embeddings.create(
        input = [text], 
        model=model)
    return res.data[0].embedding

In [ ]:
async def chat(messages:list,model:str='gpt-4o'):
    completion = await oclient.chat.completions.create(
    model=model,  # e.g. gpt-35-instant
    messages=messages,    
    )
    return completion.choices[0].message.content

In [ ]:
ADA_DIM = 1536
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=ADA_DIM,  # The vectors we will use in this demo has 768 dimensions
)

In [ ]:
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

vectors = [await embed(doc,EMB_MODEL) for doc in docs]
# The output vector has 768 dimensions, matching the collection that we just created.
print("Dim:", len(vectors[0]))  # Dim: 768 (768,)

# # Each entity has id, vector representation, raw text, and a subject label that we use
# # to demo metadata filtering later.
data = [
     {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history", "url": "http://example.com/" + str(i)}
     for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

In [ ]:
res = client.insert(collection_name="demo_collection", data=data)
print(res)

In [ ]:
question  = "Who is Alan Turing?"
query_vector = await embed(question,EMB_MODEL)
# If you don't have the embedding function you can use a fake vector to finish the demo:
# query_vectors = [ [ random.uniform(-1, 1) for _ in range(768) ] ]

res = client.search(
    collection_name="demo_collection",  # target collection
    data=[query_vector],  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject","url"],  # specifies fields to be returned
)
first_item = res[0][0]['entity']['text']
print(first_item)

In [ ]:
messages = [
    {"role": "system",      
     "content": "You are a chatbot."},
    {"role": "user", 
     "content": question+"\n"+first_item+"\n"+"Use only the provided information to answer the question."},
]
print(await chat(messages))